In [1]:
import numpy as np
import pandas as pd

# Step 1: Set a random seed for reproducibility
np.random.seed(42)

# Step 2: Define parameters
n_samples = 500      # number of data points
n_features = 7       # number of features

# Step 3: Create a covariance matrix to make features correlated
base_corr = 0.9
cov = np.full((n_features, n_features), base_corr)
np.fill_diagonal(cov, 1.0)  # diagonal = 1 (self-correlation)

# Step 4: Generate multivariate normal data (highly correlated)
mean = np.zeros(n_features)
X = np.random.multivariate_normal(mean, cov, size=n_samples)

# Step 5: Define true coefficients and bias
true_weights = np.array([2.5, -1.8, 1.2, 0.8, 0.5, 1.5, -0.7])
bias = 3.0

# Step 6: Generate target variable with some noise They define how the target variable (y) 
#is generated from your features (X) using a linear relationship + randomness
noise = np.random.normal(0, 1.5, size=n_samples)
y = X.dot(true_weights) + bias + noise

# Step 7: Create DataFrame
columns = [f'Feature_{i+1}' for i in range(n_features)]
df = pd.DataFrame(X, columns=columns)
df['Target'] = y

# Step 8: Display correlation matrix
print("Feature Correlation Matrix:")
print(df.corr())

# Step 9: Save dataset
df.to_csv("highly_correlated_dataset.csv", index=False)
print("\nDataset saved as 'highly_correlated_dataset.csv'")


Feature Correlation Matrix:
           Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  Feature_6  \
Feature_1   1.000000   0.897925   0.899279   0.895184   0.894056   0.894905   
Feature_2   0.897925   1.000000   0.904170   0.914528   0.902848   0.916318   
Feature_3   0.899279   0.904170   1.000000   0.906929   0.890779   0.892586   
Feature_4   0.895184   0.914528   0.906929   1.000000   0.902122   0.895103   
Feature_5   0.894056   0.902848   0.890779   0.902122   1.000000   0.889743   
Feature_6   0.894905   0.916318   0.892586   0.895103   0.889743   1.000000   
Feature_7   0.903157   0.907611   0.905746   0.902690   0.903508   0.895431   
Target      0.903654   0.807414   0.869086   0.854551   0.850811   0.871616   

           Feature_7    Target  
Feature_1   0.903157  0.903654  
Feature_2   0.907611  0.807414  
Feature_3   0.905746  0.869086  
Feature_4   0.902690  0.854551  
Feature_5   0.903508  0.850811  
Feature_6   0.895431  0.871616  
Feature_7   1.000000  0.82968

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# ----------------------------
# Step 1: Load the dataset
# ----------------------------
df = pd.read_csv("highly_correlated_dataset.csv")
X = df.drop("Target", axis=1).values
y = df["Target"].values.reshape(-1, 1)

# ----------------------------
# Step 2: Normalize features
# ----------------------------
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Add bias column
X = np.hstack((np.ones((X.shape[0], 1)), X))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ----------------------------
# Step 3: Ridge Regression functions
# ----------------------------
def ridge_cost(X, y, weights, lmbd):
    n = len(y)
    predictions = X.dot(weights)
    error = predictions - y
    mse = (1 / (2 * n)) * np.sum(error ** 2)
    reg = (lmbd / (2 * n)) * np.sum(weights[1:] ** 2)
    return mse + reg

def ridge_gradient(X, y, weights, lmbd):
    n = len(y)
    predictions = X.dot(weights)
    error = predictions - y
    grad = (1 / n) * (X.T.dot(error))
    grad[1:] += (lmbd / n) * weights[1:]
    return np.clip(grad, -1e3, 1e3)  # <-- prevent gradient explosion

def gradient_descent(X, y, lr, lmbd, epochs=1000):
    weights = np.zeros((X.shape[1], 1))
    for _ in range(epochs):
        grad = ridge_gradient(X, y, weights, lmbd)
        weights -= lr * grad
        if np.any(np.isnan(weights)):  # safety stop
            print(f"⚠️ NaN detected at lr={lr}, lambda={lmbd}")
            break
    return weights

# ----------------------------
# Step 4: Try different parameters
# ----------------------------
learning_rates = [0.0001, 0.001, 0.01, 0.1]   # removed 1 and 10
lambdas = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]

results = []

for lr in learning_rates:
    for lmbd in lambdas:
        weights = gradient_descent(X_train, y_train, lr, lmbd, epochs=1000)
        y_pred = X_test.dot(weights)

        if np.any(np.isnan(y_pred)):
            continue  # skip invalid runs

        cost = ridge_cost(X_test, y_test, weights, lmbd)
        r2 = r2_score(y_test, y_pred)

        results.append({
            "Learning Rate": lr,
            "Lambda": lmbd,
            "Cost": cost,
            "R2 Score": r2
        })

# ----------------------------
# Step 5: Display best result
# ----------------------------
results_df = pd.DataFrame(results)
best_row = results_df.loc[results_df["R2 Score"].idxmax()]

print("✅ Ridge Regression Optimization Complete!\n")
print("Best Parameters:")
print(best_row)
print("\nTop 5 results by R2 Score:")
print(results_df.sort_values(by="R2 Score", ascending=False).head())


✅ Ridge Regression Optimization Complete!

Best Parameters:
Learning Rate    0.100000
Lambda           1.000000
Cost             1.078937
R2 Score         0.880185
Name: 29, dtype: float64

Top 5 results by R2 Score:
    Learning Rate        Lambda      Cost  R2 Score
29            0.1  1.000000e+00  1.078937  0.880185
27            0.1  1.000000e-03  1.003463  0.879878
26            0.1  1.000000e-05  1.003385  0.879878
25            0.1  1.000000e-10  1.003385  0.879878
24            0.1  1.000000e-15  1.003385  0.879878


In [2]:
# ques 3:
# Q3 - Linear, Ridge and Lasso Regression + Cross Validation

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------------------------------
# Step 1: Load the Boston Housing Dataset
# -----------------------------------------------
df = pd.read_csv("Boston_Housing.csv")  

print("✅ Dataset Loaded Successfully!\n")
print(df.head())

# -----------------------------------------------
# Step 2: Split Input and Target
# -----------------------------------------------
X = df.drop(columns=["MEDV"], errors='ignore')  # 'MEDV' is usually the target column
y = df["MEDV"]

# -----------------------------------------------
# Step 3: Split into Train and Test sets
# -----------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------------------------
# Step 4: Feature Scaling (very important for Ridge/Lasso)
# -----------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------------------------------
# Step 5: Train Models
# -----------------------------------------------

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)

# Ridge Regression (λ = 0.5748)
ridge_reg = Ridge(alpha=0.5748)
ridge_reg.fit(X_train_scaled, y_train)
y_pred_ridge = ridge_reg.predict(X_test_scaled)

# Lasso Regression (λ = 0.5748)
lasso_reg = Lasso(alpha=0.5748)
lasso_reg.fit(X_train_scaled, y_train)
y_pred_lasso = lasso_reg.predict(X_test_scaled)

# -----------------------------------------------
# Step 6: Evaluate Each Model
# -----------------------------------------------
models = {
    "Linear Regression": (y_pred_lin, lin_reg),
    "Ridge Regression": (y_pred_ridge, ridge_reg),
    "Lasso Regression": (y_pred_lasso, lasso_reg)
}

for name, (pred, model) in models.items():
    mse = mean_squared_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    print(f"\n📊 {name}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

# -----------------------------------------------
# Step 7: RidgeCV and LassoCV (Cross Validation)
# -----------------------------------------------
alphas = np.logspace(-3, 2, 50)  # try 50 values between 10^-3 and 10^2

ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv.fit(X_train_scaled, y_train)

lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso_cv.fit(X_train_scaled, y_train)

print("\n🔍 RidgeCV Best Alpha:", ridge_cv.alpha_)
print("🔍 LassoCV Best Alpha:", lasso_cv.alpha_)

# -----------------------------------------------
# Step 8: Evaluate CV Models
# -----------------------------------------------
ridge_cv_pred = ridge_cv.predict(X_test_scaled)
lasso_cv_pred = lasso_cv.predict(X_test_scaled)

print("\n✅ RidgeCV R²:", r2_score(y_test, ridge_cv_pred))
print("✅ LassoCV R²:", r2_score(y_test, lasso_cv_pred))


✅ Dataset Loaded Successfully!

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        B  LSTAT  MEDV  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  

📊 Linear Regression
Mean Squared Error: 24.2911
R² Score: 0.6688

📊 Ridge Regression
Mean Squared Error: 24.3036
R² Score: 0.6686

📊 Lasso Regression
Mean Squared Error: 27.3059
R² Score: 0.6276

🔍 RidgeCV Best Alpha: 7.543120063354623
🔍 LassoCV Best Alpha: 0.001

✅ RidgeCV R²: 0.6666002024866851
✅ LassoCV R²: 0.6687128230636585


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


In [4]:
# Q2 - Hitters Dataset Analysis with Ridge & Lasso Regression

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

# ----------------------------------------------
# Step 1: Load the Dataset
# ----------------------------------------------
df = pd.read_csv("Hitters.csv")

print("✅ Dataset Loaded Successfully!\n")
print(df.head())
print("\nDataset Info:")
print(df.info())

# ----------------------------------------------
# Step 2: Preprocessing
# ----------------------------------------------
# (a) Handle missing values
print("\n🔹 Number of missing values before cleaning:\n", df.isnull().sum())
df = df.dropna()  # Drop rows with null values
print("\n🔹 Number of missing values after cleaning:\n", df.isnull().sum())

# (b) Encode categorical variables
# Identify categorical columns
cat_cols = df.select_dtypes(include=['object']).columns
print("\nCategorical Columns:", list(cat_cols))

# Label Encode each categorical column
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

# ----------------------------------------------
# Step 3: Split into Input and Output Features
# ----------------------------------------------
# Target variable: 'Salary'
X = df.drop("Salary", axis=1)
y = df["Salary"]

# ----------------------------------------------
# Step 4: Feature Scaling
# ----------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ----------------------------------------------
# Step 5: Train-Test Split
# ----------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ----------------------------------------------
# Step 6: Train Models
# ----------------------------------------------

# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)

# Ridge Regression
ridge_reg = Ridge(alpha=1.0)
ridge_reg.fit(X_train, y_train)
y_pred_ridge = ridge_reg.predict(X_test)

# Lasso Regression
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_train, y_train)
y_pred_lasso = lasso_reg.predict(X_test)

# ----------------------------------------------
# Step 7: Evaluate Models
# ----------------------------------------------
models = {
    "Linear Regression": y_pred_lin,
    "Ridge Regression": y_pred_ridge,
    "Lasso Regression": y_pred_lasso
}

print("\n📊 Model Evaluation Results:")
for name, preds in models.items():
    mse = mean_squared_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    print(f"\n{name}")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

# ----------------------------------------------
# Step 8: Compare and Analyze
# ----------------------------------------------
print("\n✅ Comparison Complete!")
print("Generally, Ridge performs better when features are correlated, while Lasso helps in feature selection.")


✅ Dataset Loaded Successfully!

   AtBat  Hits  HmRun  Runs  RBI  Walks  Years  CAtBat  CHits  CHmRun  CRuns  \
0    293    66      1    30   29     14      1     293     66       1     30   
1    315    81      7    24   38     39     14    3449    835      69    321   
2    479   130     18    66   72     76      3    1624    457      63    224   
3    496   141     20    65   78     37     11    5628   1575     225    828   
4    321    87     10    39   42     30      2     396    101      12     48   

   CRBI  CWalks League Division  PutOuts  Assists  Errors  Salary NewLeague  
0    29      14      A        E      446       33      20     NaN         A  
1   414     375      N        W      632       43      10   475.0         N  
2   266     263      A        W      880       82      14   480.0         A  
3   838     354      N        E      200       11       3   500.0         N  
4    46      33      N        E      805       40       4    91.5         N  

Dataset Info:
<cla

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.530e+06, tolerance: 4.367e+03
  model = cd_fast.enet_coordinate_descent(


In [5]:
# Q4 - Multiclass Logistic Regression (One-vs-Rest) from scratch

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ----------------------------------------------------
# Step 1: Load the Iris dataset
# ----------------------------------------------------
iris = load_iris()
X = iris.data        # 4 features
y = iris.target      # 3 classes: 0,1,2

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Add bias term
X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])

# ----------------------------------------------------
# Step 2: Define sigmoid & gradient descent functions
# ----------------------------------------------------
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, weights):
    n = len(y)
    h = sigmoid(X.dot(weights))
    epsilon = 1e-9  # to avoid log(0)
    cost = (-1 / n) * (y.T.dot(np.log(h + epsilon)) + (1 - y).T.dot(np.log(1 - h + epsilon)))
    return cost

def gradient_descent(X, y, lr=0.1, epochs=1000):
    weights = np.zeros((X.shape[1], 1))
    for _ in range(epochs):
        h = sigmoid(X.dot(weights))
        gradient = (1 / len(y)) * X.T.dot(h - y)
        weights -= lr * gradient
    return weights

# ----------------------------------------------------
# Step 3: One-vs-Rest Training
# ----------------------------------------------------
num_classes = len(np.unique(y_train))
weights_all = []

for c in range(num_classes):
    # Create binary labels for class c
    y_binary = (y_train == c).astype(int).reshape(-1, 1)
    weights_c = gradient_descent(X_train, y_binary, lr=0.1, epochs=2000)
    weights_all.append(weights_c)

weights_all = np.hstack(weights_all)  # shape: (features, classes)

# ----------------------------------------------------
# Step 4: Prediction
# ----------------------------------------------------
def predict(X, weights_all):
    probs = sigmoid(X.dot(weights_all))  # get probabilities for each class
    preds = np.argmax(probs, axis=1)
    return preds

y_pred = predict(X_test, weights_all)

# ----------------------------------------------------
# Step 5: Evaluation
# ----------------------------------------------------
acc = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy on test data: {acc*100:.2f}%\n")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))


✅ Accuracy on test data: 90.00%

Confusion Matrix:
[[10  0  0]
 [ 0  8  2]
 [ 0  1  9]]

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.89      0.80      0.84        10
   virginica       0.82      0.90      0.86        10

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

